# Die Vermessung der medienwissenschaftlichen Welt?
## Datengestützte Analysen mit media/rep/

Dieses Jupyter Notebook soll eine Grundlage bieten, die im Fachrepositorium media/rep/ vorhandenen Texte und die dazugehörigen Metadaten mithilfe eines Python-Skriptes zu analysieren. Hierzu werden Abschnitte für die Vorbereitung von Texten zur Analyse (Entfernung von Stoppwörtern, Neuzusammenstellung des Textkorpus, Gruppierung von Texten nach Jahreszahlen) sowie ein Durchsuchen der Texte nach konkreten Begriffen (als vollständige Wörter oder in Form einer Wildcard-Suche, Identifizierung von einzelnen Wortformen und -kombinationen bei einer Wildcard-Suche, Identifizierung von Einzeltexten, die einen Suchbegriff vermehrt enthalten) bereitgestellt. Funktional orientiert sich dieses Jupyter Notebook damit an Textanalysetools wie Voyant.
  
Generell eignet sich der bereit gestellte Code auch für die Analyse anderer Texte, die in demselben Format wie das media/rep/-Korpus vorliegen, wobei jedoch an einigen Stellen Anpassungen notwendig sein dürften.
  
Um das Notebook nutzen zu können, sind Grundkenntnisse in Python und im Umgang mit Jupyter Notebooks notwendig. Als ein Ergebnis aus der Zusammenarbeit von Medienwissenschaft und Digital Humanities ist das Notebook nicht daraufhin überarbeitet worden, eine möglichst hohe Performanz in der Umsetzung der Analyseschritte zu erreichen. Die Kommentierung der einzelnen Code-Abschnitte dient vor allem dazu, transparent zu machen, welche Schritte durchgeführt wurden, um die Analyseergebnisse zu erzielen.

### Inhalt  
1. [Vorbedingungen](#prerequisites)
1. [Textpreprocessing](#preprocessing)  
   a. [Eingrenzung des Korpus](#eingrenzung)    
   b. [Gruppierung von Texten nach Publikationsjahr](#gruppierung)  
   c. [Entfernen von Stoppwörtern](#stoppwörter)  
2. [Textanalyse](#textanalyse)  
   a. [Suche nach vollständigen Wörtern oder Wildcard-Suche](#ganzewörter)  
   b. [Identifizierung von verschiedenen Wortformen und -kombinationen einzelner Suchbegriffe bei einer Wildcard-Suche](#wortformen)  
   c. [Identifizierung von Einzeltexten, die einen Suchbegriff enthalten](#einzeltexte)  

## Vorbedingungen <a class="anchor" id="prerequisites"></a>

Die Analysegrundlage für dieses Jupyter Notebook stellen die im Fachrepositorium media/rep/ vorliegenden Publikationen dar. Diese können mithilfe eines Python-Skriptes im txt-Format heruntergeladen werden, wobei gleichzeitig eine Metadatentabelle generiert wird. Dieses Skript wird separat von diesem Jupyter Notebook zur Verfügung gestellt.
   
Für die Nutzung der in diesem Jupyter Notebook zur Verfügung gestellten Funktionalitäten ist es notwendig, die mittels des Skripts extrahierten txt-Dateien in einem Dateiordner zu speichern, der wiederum in einem übergeordneten Verzeichnis liegt, in der auch die bei Abruf der txt-Dateien automatisch gespeicherte Metadatentabelle sowie dieses Jupyter Notebook gespeichert werden. Hinzu kommt noch eine Stoppwortliste, die im Rahmen des Textpreprocessing verwendet wird und hier ebenfalls separat zur Verfügung gestellt wird. Diese vorgeschlagene Struktur gilt sowohl für die Nutzung des Jupyter Notebooks in einer IDE als auch einer Jupyter Umgebung. Die Ordnerstruktur sollte dementsprechend wie folgt aussehen:  
  
Hauptverzeichnis  
-- > Dateiordner mit txt-Dateien  
------- > 1988_....txt  
------- > 2013_....txt  
------- > ...  
------- > 2020_....txt    
-- > Metadatentabelle.csv  
-- > Jupyter Notebook.ipynb  
-- > Stoppwortliste.txt  
  
Die Benennung der aus media/rep/ extrahierten txt-Dateien sollte grundsätzlich beibehalten werden, da einige Schritte des Textpreprocessings an die automatisch generierten Dateinamen angepasst sind. Die Benennung des Dateiordners mit den txt-Dateien sowie die Benennung der Metadatentabelle sind frei wählbar und werden an gekennzeichneten Stellen in diesem Jupyter Notebook abgefragt, um die Analyseschritte darauf durchführen zu können.
  
Generell empfiehlt es sich, eine Kopie der aus media/rep/ extrahierten Texte zu behalten, da durch das Textpreprocessing Inhalte verändert werden können. Um den Zustand der einzelnen Zwischenschritte des Preprocessing zu speichern, werden in den nachfolgenden Skripten automatisch neue Dateiordner generiert, in dem die bearbeiteten Dateien gespeichert werden.
  
Der in diesem Jupyter Notebook zur Verfügung gestellte Code wurde mit den Python-Versionen 3.8 - 3.11 getestet.
Zur Nutzung aller Funktionalitäten des Notebooks ist die Installation und der Import folgender Libraries notwendig:

In [1]:
#Textbereinigung
import re
#Einlesen der txt-Dateien
import os
import glob
from pathlib import Path
import shutil
#Erstellung von Dataframes und Berechnungen auf diesen
import pandas as pd
#Visualisierungen (Styling und Darstellung)
import pygal 
from pygal.style import Style
import cairosvg
import lxml
import tinycss2
import cssselect2
#Wortwolken
import matplotlib.pyplot as plt
from wordcloud import WordCloud

Das Jupyter Notebook ist vom Prinzip her so aufgebaut, dass dem eigentlichen Skript jeweils eine Code-Zelle vorgelagert ist, in der beispielsweise Angaben zu dem Ordnernamen, der die zu analysierenden Dateien enthält, zu Suchbegriffen oder der Jahresrange der zu analysierenden Texte gemacht werden müssen.
  
Aufgrund der Größe des media/rep/-Textkorpus kann das Durchlaufen einer Code-Zelle je nach Bearbeitungsschritt bis zu 4 Minuten betragen.

## Textpreprocessing <a class="anchor" id="preprocessing"></a>

### Eingrenzung des Korpus <a class="anchor" id="eingrenzung"></a>

Das Herunterladen von txt-Dateien aus media/rep/ lässt sich bereits über das oben erwähnte Skript den Analysebedürfnissen entsprechend anpassen, indem nur bestimmte Teile des Korpus extrahiert bzw. nicht extrahiert werden. Aber auch nach Herunterladen des Gesamtkorpus besteht die Möglichkeit, dieses in kleinere Teil-Korpora zu zerlegen, um gezielt Analysen auf beispielsweise allen publizierten Rezensionen oder auf allen Ausgaben einer spezifischen Zeitschrift eines bestimmten Zeitraums durchzuführen. Dies kann durch Auswählen oder Ausschließen eines bestimmten Teils des Korpus geschehen. In dem eingangs erwähnten Artikel wurde für die Analyse beispielsweise die Publikationsform PeriodicalPart nicht mitberücksichtigt, da darin einige Publikations-Items vorliegen, die gleichzeitig als review oder article geführt werden und somit doppelt in die Analyse eingeflossen wären. Dies gilt vereinzelt auch für book und bookPart.

media/rep/ bietet unter anderem folgende Publikationsformen: review, article, PeriodicalPart, book, bookPart, workingPaper, doctoralThesis, report

Des Weiteren sind verschiedene Publikationsreihen - sowohl Buch als auch Zeitschrift - vorhanden, die über eine ISSN-Nummer identifiziert werden können. Beispielsweise ist die Zeitschrift 'MEDIENwissenschaft: Rezensionen | Reviews' mit der 'issn:2196-4270' verknüpft. Einen Überblick über die Publikationsreihen und ihre jeweiligen ISSN-Nummern bietet die beim Herunterladen der txt-Dateien automatisch generierte Metadatentabelle.

Als weitere Filtermöglichkeiten bietet sich zudem die Auswahl bestimmter Publikationsjahre oder Autor*innen an. Auch hierfür bietet die Metadatentabelle einen guten Überblick.

Um die nachfolgenden Schritte durchführen zu können, dürfen die Dateinamen der aus media/rep/ extrahierten Dateien nicht nachträglich verändert worden sein, sondern müssen in ihrem Originalzustand belassen werden. Dies gilt auch für die in der Metadatentabelle aufgeführten Dateinamen.

In [ ]:
'''
Einzufügen ist der Name der aus media/rep/ gewonnenen Metadaten-Datei, 
die dann in einen Dataframe eingelesen wird.
'''
metadata = "" #z.B. "metadata.csv"

df_metadata = pd.read_csv(metadata, low_memory=False)

'''
Nachfolgend sind die verschiedenen Filteroptionen gelistet, 
die durch Entkommentieren ausgewählt werden können. 
Es besteht die Möglichkeit, Parameter auszuschließen (excluded) 
oder gezielt auszuwählen (included).
Die ausgewählten Parameter können in die jeweils leere Liste, 
die .isin([ ]) folgt, eingetragen werden.
Für jede Filteroption wird ein Beispiel gezeigt.
'''

'''
Option 1: Filtern nach Erscheinungsjahr
Wählt oder schließt die Publikationen aus, 
die in den in der Liste eingetragenen Erscheinungsjahren publiziert wurden.
Beispiel: 
years_excluded = df_metadata['dc.date.issued'].isin(["2012", "2013", "2014"]) == False 
schließt alle Publikationen aus, die in den genannten Jahren publiziert wurden.
Das Erscheinungsjahr bezieht sich nicht auf die Publikation in media/rep/, 
sondern auf die Erstpublikation.
Die ersten drei Code-Zeilen müssen unverändert übernommen werden. 
Da die in media/rep/ enthaltenen Jahresangaben 
unterschiedlichen Formaten folgen, müssen diese angeglichen werden.
'''
#years = df_metadata["dc.date.issued"].to_list()
#years = [i[:4] for i in years]
#df_metadata["dc.date.issued"] = years
#years_excluded = df_metadata['dc.date.issued'].isin([]) == False
#years_included = df_metadata['dc.date.issued'].isin([]) 

'''
Option 2: Filtern nach Publikationsformat
Wählt oder schließt die Publikationen aus, 
die in den in der Liste eingetragenen Publikationsformaten publiziert wurden.
Beispiel: 
types_included = df_metadata['dc.type'].isin(["article", "PeriodicalPart"]) 
wählt alle Publikationen aus, 
die als article und PeriodicalPart publiziert wurden.
'''
#types_excluded = df_metadata['dc.type'].isin([]) == False
#types_included = df_metadata['dc.type'].isin([])

'''
Option 3:
Wählt oder schließt die Publikationen aus, 
die in den in der Liste eingetragenen Publikationsreihen publiziert wurden.
Beispiel: 
isPartof_excluded = df_metadata['dc.relation.isPartOf'].isin(["issn:2196-4270"]) == False 
schließt alle Publikationen der Zeitschrift "MEDIENwissenschaft: Rezensionen | Reviews" aus.
'''
#isPartof_excluded = df_metadata['dc.relation.isPartOf'].isin([]) == False
#isPartof_included = df_metadata['dc.relation.isPartOf'].isin([])

'''
Option 4:
Wählt oder schließt die Publikationen aus, 
die von bestimmten Autor*innen verfasst wurden.
Beispiel: 
creator_included = df_metadata['dc.creator'].isin(["Kessler, Frank"])
wählt nur Publikationen aus, 
die von genanntem Autor verfasst wurden.
'''
#creator_excluded = df_metadata['dc.creator'].isin([]) == False
#creator_included = df_metadata['dc.creator'].isin([])

'''
Um mehrere Filteroptionen gemeinsam nutzen zu können, 
müssen diese mit ihren oben genannten Variablennamen in der gewünschten 
Reihenfolge des Filterns in die nachstehende Liste eingetragen werden.
Beispielsweise ließen sich somit zuerst alle article auswählen 
und in einem zweiten Schritt die Auswahl auf bestimmte Jahre eingrenzen.
Die einzelnen Variablen müssen dabei mit dem & Zeichen verbunden werden.
'''
filter = #z.B. types_excluded & years_excluded

'''
Einzufügen ist der Name des Ordners, 
in dem sich alle aus media/rep/ extrahierten Texte befinden.
'''
folder = "" #z.B. "fulltext"

'''
Einzufügen ist der Name des Ordners, 
in dem die ausgewählten Dateien gespeichert werden sollen 
(dieser wird automatisch in diesem Verzeichnis generiert).
'''
filtered_folder = "" #z.B. "filtered_corpus"

os.mkdir(filtered_folder)

In [12]:
'''
Der Dataframe wird nach den ausgewählten Optionen gefiltert.
'''
df_metadata_filtered = df_metadata.loc[filter]

'''
Die Dateinamen aller übrig gebliebenen Einträge im Dataframe 
werden in eine Liste überführt.
'''
filenames_list = df_metadata_filtered["fulltext_file"].to_list()

'''
Die Dateinamen in der Liste werden mit den Dateinamen 
der im Ordner vorhandenen txt-Dateien abgeglichen.
Wenn ein Dateiname im Ordner auf der Liste zu finden ist, 
wird diese Datei in den neu erstellten Ordner kopiert, 
der abschließend das neu erstellte Textkorpus enthält.
'''
for filename in os.listdir(folder):
    if filename in filenames_list:
        full_file_path = os.path.join(folder, filename)
        shutil.copy((os.path.join(folder, filename)), filtered_folder)

#### Gruppierung von Texten nach Publikationsjahr <a class="anchor" id="gruppierung"></a>

Ein Gesamtabzug des media/rep/-Korpus besteht aus einer Menge von über 18.000 Texten (Stand Januar 2023). Abhängig von den jeweiligen Analysefragen können diese Texte separat betrachtet werden. Insbesondere für eine Visualisierung von Ergebnissen ist es jedoch notwendig, diese Dateien zu gruppieren. Hierzu bietet sich eine Gruppierung nach Publikationsjahr an. Hierbei ist nicht das Jahr der Publikation in media/rep/, sondern das Jahr der Erstpublikation relevant.

In [ ]:
'''
Einzufügen ist der Name des Ordners, 
in dem sich alle aus media/rep/ extrahierten Texte befinden.
'''
folder = "" #z.B. "fulltext"

'''
Einzufügen ist der Name des Ordners, 
in dem die neu zusammengeführten Dateien gespeichert werden sollen 
(dieser wird automatisch in diesem Verzeichnis generiert).
Es ist sinnvoll, diese nach Jahreszahlen gruppierten txt-Dateien 
in einem separaten Ordner zu speichern, 
da diese ansonsten inmitten der zehntausenden Dateien 
des bereits vorhandenen Ordners gespeichert würden 
und manuell herausgesucht werden müssten.
'''
year_folder = "" #z.B. "fulltext_by_year"

os.mkdir(year_folder)

In [ ]:
'''
Die vorhandenen Jahreszahlen werden 
aus den Dateinamen im gewählten Ordner extrahiert 
und in eine Liste überführt, 
wobei Duplikate entfernt 
und die Liste aufsteigend sortiert wird.
'''
years = [year for file in os.listdir(folder) for year in re.findall("(\d{4})_", file)]
years = list(dict.fromkeys(years))
years.sort()

'''
Die txt-Dateien, die zusammengeführt werden sollen, 
werden zusammen gemäß ihrem Publikationsjahr 
aus dem enstprechenden Ordner eingelesen.
'''
for year in years:
    files = glob.glob(os.path.join(f"{folder}", f"{year}_*.txt"))                   
    '''
    Die nach Jahren eingelesenen Texte werden in dem neu erstellten Ordner
    als eine gemeinsame txt-Datei pro Jahr erstellt.
    '''
    with open(os.path.join(f"{year_folder}", f"{year}_fulltext.txt"), "wb") as outfile:
        for f in files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

### Entfernen von Stoppwörtern <a class="anchor" id="stoppwörter"></a>

Um eine Textanalyse durchzuführen, bietet es sich an, vorab Stoppwörter zu entfernen, die für die Analyse nicht relevant sind (z.B. Präpositionen, Artikel, Füllwörter). Aufgrund der Größe des media/rep/-Korpus wird die inhaltliche Analyse der Texte durch das Entfernen von Stoppwörtern beschleunigt und erlaubt eine Fokussierung auf für das Textverständnis bedeutende Wörter. Dies ist insbesondere dann notwendig, wenn die inhaltliche Analyse das Ermitteln der häufigsten (semantisch bedeutsamen) Wörter beinhaltet. Im Rahmen der hier vorgestellten Analyse ist die Entfernung von Stoppwörtern zwar zu empfehlen, aber nicht zwingend erforderlich, da einer deduktiven Logik gefolgt und eine gezielte Suche nach Begriffen ermöglicht wird. Dies steht einem induktiven Ansatz gegenüber, bei dem durch Ermittlung von Worthäufigkeiten aller Wörter der Texte nach den häufigsten Begriffen gesucht wird. Letzteres stellt aufgrund der Größe des media/rep/-Korpus eine Herausforderung dar und erfordert eine zusätzliche Strategie, was den Umgang mit Stoppwörtern und der gemeinsamen Analyse verschiedener Wortformen angeht, und bietet sich somit für eine eigene Untersuchung an.
  
Zur Entfernung von Stoppwörtern kann auf eine im Kontext dieses Projektes erstellte Stoppwortliste zugegriffen werden, die auf der deutschen und englischen Stoppwortliste des Textanalysetools Voyant basiert und zusätzlich spezifisch dem media/rep/-Korpus angepasste Begriffe enthält.
  
Diese txt-Datei und somit die Auswahl und Anzahl der Stoppwörter kann je nach Bedarf angepasst werden. Vorraussetzung für die nachfolgenden Schritte ist, dass die Stoppwortliste in demselben Verzeichnis gespeichert ist wie dieses Jupyter Notebook.

In [ ]:
'''
Einzufügen ist der Name der Stoppwortliste.
'''
stopword_list = "" #z.B. "stopwords.txt"

'''
Einzufügen ist der Name des Ordners, 
in dem sich alle aus media/rep/ extrahierten Texte befinden.
'''
folder = "" #z.B. "fulltext_by_year"

'''
Einzufügen ist der Name des Ordners, 
in dem die nach Stoppwörtern gefilterten Dateien gespeichert werden sollen 
(dieser wird automatisch in diesem Verzeichnis generiert).
'''
stopword_folder = "" #z.B. "stopwordfiltered_by_year"

os.mkdir(stopword_folder)

In [ ]:
'''
Einlesen der txt-Datei mit der Stoppworliste
und Überführen der Stoppwörter in eine Liste.
'''
with open(stopword_list, "r", encoding="utf-8") as infile1:
    stopwords = [word.rstrip() for word in infile1]

'''
Jede im Dateiordner vorliegende txt-Datei 
wird nacheinander eingelesen 
und verschiedene Textbereinigungsschritte durchgeführt.
'''
for file in glob.glob(os.path.join(f"{folder}", "*.txt")):
    with open(file, "r", encoding="utf-8") as infile2:
        text_stop = infile2.read()
        '''
        Am Ende einer Zeile durch Worttrennung 
        voneinander getrennte Wörter 
        werden zusammengeführt. 
        Zahlen, Zeilenumbrüche und Nicht-Wortzeichen 
        werden entfernt 
        und durch ein Leerzeichen ersetzt.
        '''
        text_stop = re.sub("-\n+|\d", "", text_stop)
        text_stop = re.sub("\n|\W+", " ", text_stop)
        
        '''
        Die Stoppwörter werden aus den Texten entfernt, 
        indem die Wörter eines Textes 
        jeweils von einem String in eine Liste überführt 
        und die Stoppwörter aus dieser Liste herausgefiltert werden. 
        Anschließend wird die übriggebliebene Wortliste 
        eines jeden Textes wieder in einen String überführt. 
        ''' 
        text_stop = ' '.join([word for word in text_stop.split() if word.lower() not in stopwords])

        '''
        Die Dateinamen der eingelesenen Texte 
        werden auf ihren Kernnamen reduziert 
        (Dateierweiterung, Dateipfad wird entfernt).
        '''
        filename = Path(file).stem

        '''
        Die nach Stoppwörtern gefilterten Texte 
        werden in dem neu erstellten Ordner gespeichert.
        '''
        with open(os.path.join(f"{stopword_folder}", f"{filename}.txt"), "w", encoding="utf-8") as outfile:
            output_text = outfile.write(text_stop)

## Textanalyse <a class="anchor" id="textanalyse"></a>

Der folgende Abschnitt bezieht sich auf die Suche nach Wörtern in dem vorab erstellten Textkorpus und einer Visualisierung von Ergebnissen. Zum einen wird eine Suche nach vollständigen Wörtern bereitgestellt, zum anderen eine Wildcard-Suche, bei der lediglich der Anfang eines Wortes eingegeben werden muss und alle verschiedenen Endungsmöglichkeiten des Wortes in der Suche mitberücksichtigt werden. Die Ergebnisse werden in einen Dataframe überführt und können als csv-Datei gespeichert werden oder in Form eines Liniendiagramms betrachtet werden. 
  
Um bei der Häufigkeitsanalyse die unterschiedliche Länge von Publikationen mitberücksichtigen zu können, werden relative statt absolute Zahlen in den Ergebnissen präsentiert. Dies bedeutet, dass die Häufigkeit des Vorkommens eines Suchbegriffes in einem Text in Relation zu der Gesamtzahl der Wörter eines Textes gesetzt wird. Abhängig davon, ob während des Textpreprocessing Stoppwörter aus den Texten entfernt wurden oder nicht, steht die ermittelte Anzahl der Wörter eines vorbearbeiteten Textes nicht in jedem Fall für die eigentliche Gesamtlänge eines Textes. Wurden die Stoppwörter vorab entfernt, sind die relativen Werte höher, als wenn sie in Bezug auf die Gesamtlänge eines unbereinigten Textes berechnet würden. Ein isoliert betrachteter Wert ist daher nicht aussagekräftig. Erst die Kontextualisierung und der Vergleich mit weiteren Werten lässt Schlussfolgerungen über die Häufigkeit der Suchbegriffe zu.
  
Um eine übersichtliche Ergebnismenge zu generieren, bedarf es insbesondere hier einer Gruppierung der ursprünglich mehr als 18.000 Texte aus media/rep/. Hierzu bietet sich die Gruppierung nach Jahren an, wobei insbesondere der Zeitraum zwischen 1980 und 2020 eine repräsentative Textbasis für die Analyse bietet.

### Suche nach vollständigen Wörtern oder Wildcard-Suche <a class="anchor" id="ganzewörter"></a>

In [13]:
'''
In die Liste keywords können die Suchbegriffe eingefügt werden. 
Diese müssen kleingeschrieben werden.
Es besteht die Möglichkeit einen vollständigen Begriff einzugeben, 
der in exakt dieser Form gesucht wird. 
Beispiel: 
Der Suchbegriff "digital" würde nicht die Vorkommen von 
"digitale", "digitales" etc. berücksichtigen, 
sondern nur den Begriff exakt wie er geschrieben wird.
Es besteht aber auch die Möglichkeit 
nur die Anfangsbuchstaben eines Begriffes einzugeben, 
um verschiedene Wortformen und -kombinationen mitzuberücksichtigen 
Beispiel: 
Der Suchbegriff "digital" würde alle Wörter berücksichtigen, 
bei denen "digital" den Beginn des Wortes darstellt: 
"digitales", "digitalität", "digitalpakt" etc.
Ob eine Suche nach vollständigen Wörtern 
oder eine Wildcard-Suche gewünscht ist, 
muss in der nachfolgenden Code-Zelle 
an der gekennzeichneten Stelle gewählt werden.
Prinzipiell können so viele Suchbegriffe eingegeben werden, 
wie gewünscht ist. 
Zu viele Begriffe bedeuten jedoch auch eine größere Anzahl von Linien
im erstellten Diagramm, was zu Unübersichtlichkeit führen kann. 
Es ist daher empfehlenswert, maximal 6 Suchbegriffe zu wählen.
'''
keywords = [] # z.B. ["virtuell", "internet", "netz", "online", "interaktiv", "digital"]

'''
Einzufügen ist der Name des Dateiordners, 
der die zu analysierenden txt-Dateien enthält.
'''
folder = "" #z.B. stopwordfiltered_by_year

'''
Einzufügen ist die Range der untersuchten Texte als Index 
für den zu erstellenden Dataframe mit den Analyseergebnissen 
sowie für die Beschriftung der Visualisierung.
Dies ist abhängig von der untersuchten Textgrundlage. 
Sind die Texte beispielsweise nach Jahren gruppiert 
und stammen aus den Jahren 1980-2020 
ist die range(1980, 2021, 1) zu wählen 
(zur Erklärung der range-Funktion siehe
https://docs.python.org/3/library/functions.html#func-range).
'''
range_texts = range(1980, 2021, 1)

'''
Einzufügen ist der Dateiname für die Speicherung der Ergebnisse als csv-Datei
'''
name_csv = "" #z.B. "suche_digit_virtu"

'''
Einzufügen ist der Dateiname für die Speicherung der Ergebnisdiagramme (png, svg)
'''
name_chart = "" #z.B. "suche_digit_virtu"

BITTE BEACHTEN: Für die Auswahl zwischen einer Suche nach vollständigen Wörtern und einer Wildcard-Suche müssen im folgenden Abschnitt an gekennzeichneter Stelle Code-Zeilen auskommentiert bzw. entkommentiert werden.

In [ ]:
'''
Anlegen eines leeren Dataframe, 
um die Ergebnisse der Häufigkeitsermittlung darin festzuhalten.
'''
df_results = pd.DataFrame()

'''
Anlegen einer leeren Liste, 
in der die Gesamtzahl der Wörter der einzelnen Texte gespeichert wird 
(relevant für die Berechnung von relativen Häufigkeiten).
'''
number_words = []

'''
Anlegen eines Dictionary, 
in dem die Suchbegriffe jeweils als key gespeichert werden 
und als value eine leere Liste angelegt wird, 
in der im Folgenden die Häufigkeit des Suchbegriffs 
in den einzelnen Texten eingefügt wird.
'''
frequency_keywords = {key:[] for key in keywords}

'''
Jede im Dateiordner vorliegende Textdatei 
wird nacheinander eingelesen 
und die Texte zur besseren Vergleichbarkeit 
vollständig in Kleinbuchstaben umgewandelt.
'''
for file in glob.glob(os.path.join(f"{folder}", "*.txt")):
    with open(file, "r", encoding="utf-8") as infile:
        fulltext = infile.read().lower()

        '''
        Jeder Text wird von einem String in eine Wortliste umgewandelt, 
        wobei die Wörter an den Nicht-Wortzeichen gesplittet werden.
        '''
        fulltext_list = re.split("\W+", fulltext)

        '''
        Die Länge der jeweils erstellten Wortlisten gibt Auskunft über die Länge des Textes 
        und wird für jeden Text einer Liste hinzugefügt.
        '''
        number_words.append(len(fulltext_list))

        '''
        Erstellen eines Dictionary für jeden Text.
        Hierzu wird über die Wortliste eines jeden Textes iteriert 
        und die gefundenen Suchbegriffe als keys dem Dictionary hinzugefügt. 
        Bei erstmaligem Vorkommen erhält der Suchbegriff 1 als value.
        Taucht der Suchbegriff mehrfach im Text auf, 
        wird der value für jedes Vorkommen hochgezählt.
        '''
        fulltext_dict = {}
        for i in fulltext_list:
                if i in fulltext_dict:
                    fulltext_dict[i] +=1
                else:
                    fulltext_dict[i] = 1
        
        '''
        Aus den für die einzelnen Texte erstellen Dictionaries 
        werden nur die gesuchten Begriffe ausgewählt.
        Die Anzahl des Vorkommens der einzelnen Suchbegriffe wird pro Text summiert. 
        Die summierten Ergebnisse werden in Form einer Liste 
        als value dem eingangs erstellten Dictionary hinzugefügt.
        '''
        
        '''
        EINE DER BEIDEN FOLGENDEN OPTIONEN WÄHLEN UND DIE ANDERE AUSKOMMENTIEREN:
        OPTION 1: SUCHE NACH VOLLSTÄNDIGEN WÖRTERN
        '''
        # for i in keywords:
        #     keywords_dict = {k: v for k, v in fulltext_dict.items() if k == i}
        #     frequency_keywords[i].append(sum(keywords_dict.values()))

        '''
        OPTION 2: WILDCARD_SUCHE
        '''
        # for i in keywords:
        #    keywords_dict = {k: v for k, v in fulltext_dict.items() if k.startswith(i)}
        #    frequency_keywords[i].append(sum(keywords_dict.values()))

'''
Die Ergebnisse aus dem Dictionary 
sowie die Anzahl der Wörter pro Text 
werden in den Dataframe eingefügt. 
Die eingangs definierte Text-Range 
dient als Index des Dataframe.
'''
df_results.index = range_texts
for k,v in frequency_keywords.items():
   df_results[k] = v
df_results["words"] = number_words

'''
Um relative Werte zu erhalten, 
wird die Anzahl des Vorkommens der keywords 
durch die Gesamtzahl der Wörter eines Textes dividiert.
Für eine intiuitivere Erfassung der entstandenen Werte 
wird das Ergebnis mit 1000 multipliziert, 
so dass damit die Häufigkeit des Vorkommens eines Suchbegriffs 
pro 1000 Wörter Text dargestellt werden kann.
Die Kolumne mit der Gesamtzahl der Wörter eines Textes 
wird anschließend gelöscht, 
da sie für die Erstellung der Visualisierung irrelevant ist.
'''
for column in df_results:
    df_results[column] = df_results.loc[:,column].div(df_results.loc[:,"words"], axis = 0) * 1000
df_results = df_results.drop(['words'], axis = 1)

'''
Speichern der Ergebnisse in einer csv-Datei
'''
df_results.to_csv(f"{name_csv}.csv")


Zusätzlich zur vorangehenden Code-Zelle kann die nachfolgende Code-Zelle ausgeführt werden, um die Ergebnisse in Form eines Liniendiagramms darzustellen oder zu speichern.

In [ ]:
'''
Festlegen der Parameter für die Visualisierung 
(Farbe der Kurven, Schriftart und -größe für die Beschriftung des Diagramms, 
Strichstärke für Diagrammlinien)
'''
diagram_style = Style(
    colors=('#FF0000', '#00FF00', '#0000FF', '#8A2BE2', '#000000', '#FFCC33'),
    font_family='googlefont:Anuphan',
    title_font_size=24,
    label_font_size=13,
    major_label_font_size=13,
    guide_stroke_dasharray=0,
    major_guide_stroke_dasharray=0)

'''
Erstellen eines Liniendiagramms aus den im Dataframe ermittelten Werten 
(für den Hintergrund der kubischen Interpolation 
siehe https://de.wikipedia.org/wiki/Spline-Interpolation)
'''
linechart_results = pygal.Line(style = diagram_style, x_label_rotation = 60, interpolate = "cubic")
linechart_results.title = "Häufigkeit des Vorkommens der analysierten Begriffe pro 1000 Wörter des Gesamttexts"
linechart_results.x_labels = range_texts
for column in df_results:
    linechart_results.add(f'{column}', df_results.loc[:,column])

'''
Darstellen des Liniendiagramms
'''
linechart_results


#Zum Speichern des erstellten Diagramms als svg oder png 
#können die nachfolgenden Code-Zeilen entkommentiert werden.

#linechart_results.render_to_file(f"{name_chart}.svg")
#linechart_results.render_to_png(f"{name_chart}.png")

### Identifizierung von verschiedenen Wortformen und -kombinationen einzelner Suchbegriffe bei einer Wildcard-Suche <a class="anchor" id="wortformen"></a>

Durch die oben beschriebene Wildcard-Suche wird nicht ersichtlich, welche Wortformen und -kombinationen für die einzelnen Suchbegriffe berücksichtigt werden. Das nachfolgende Skript soll die Möglichkeit bieten, diese einzelnen Formen zu identifizieren, um eine stärkere Transparenz über die Ergebnisse der Wortzählung zu erzielen. Hierzu ist immer nur ein Suchbegriff auf einmal einzugeben (keine Liste von Suchbegriffen).

In [2]:
'''
Einzufügen ist das Suchwort, 
für das die ermittelten Einzelformen ausgegeben werden sollen. 
Beispiel:
Die Suche nach "postkol" würde die Anzahl an Einzelformen 
wie Postkolonialismus, postkolonialistisch, postkolonial wiedergeben.
'''
keyword = "" #z.B. "postkol"

'''
Einzufügen ist der Name des Dateiordners, 
der die zu analysierenden txt-Dateien enthält.
'''
folder = "" #z.B. stopwordfiltered_by_year

In [7]:
'''
Anlegen einer leeren Liste, 
in der die im Weiteren erstellten Dictionaries gesammelt werden.
'''
list_dict_keyword = []

'''
Jede im Dateiordner vorliegende txt-Datei wird nacheinander eingelesen
und die Texte zur besseren Vergleichbarkeit vollständig in Kleinbuchstaben umgewandelt.
'''
for file in glob.glob(os.path.join(f"{folder}", "*.txt")):
    with open(file, "r", encoding="utf-8") as infile:
        text_keyword = infile.read().lower()
        
        '''
        Jeder Text wird von einem String in eine Wortliste umgewandelt, 
        wobei die Wörter an den Nicht-Wortzeichen gesplittet werden.
        '''
        text_keyword_list = re.split("\W+", text_keyword)

        '''
        Erstellen eines Dictionary für jeden Text.
        Hierzu wird über die Wortliste eines jeden Textes iteriert 
        und die gefundenen Suchbegriffe als keys dem Dictionary hinzugefügt.
        Bei erstmaligem Vorkommen erhält der Suchbegriff 1 als value.
        Taucht der Suchbegriff mehrfach im Text auf, 
        wird der value für jedes Vorkommen hochgezählt.
        '''
        keyword_dict = {}
        for i in text_keyword_list:
                if i in keyword_dict:
                    keyword_dict[i] +=1
                else:
                    keyword_dict[i] = 1
        
        '''
        Aus den für die einzelnen Texte erstellen Dictionaries 
        werden nur die Wörter ausgewählt, 
        die mit dem als keyword ausgewählten Suchbegriff beginnen.
        '''
        keyword_dict = {k: v for k, v in keyword_dict.items() if k.startswith(keyword)}
        
        '''
        Die für jeden Text entstandenen Dictionaries werden einer Liste hinzugefügt.
        '''
        list_dict_keyword.append(keyword_dict)

'''
Die in der Liste gesammelten einzelnen Dictionaries 
werden gemäß ihres keys (=der gefundenen Wortform) 
in einem gemeinsamen Dictionary zusammengefügt, 
wobei die Anzahl des Vorkommens der Wortform (= value des Dictionaries) 
in den einzelnen Texten zuerst als Liste gespeichert wird.
'''
keyword_dict_complete = {
    k: [d.get(k) for d in list_dict_keyword if k in d]
    for k in set().union(*list_dict_keyword)
}

'''
Die in den values des Dictionary als Liste gespeicherten 
Einzelwerte des Vorkommens der einzelnen Wortformen werden summiert.
'''
for k,v in keyword_dict_complete.items():
    keyword_dict_complete[k] = sum(v)

'''
Das Dictionary wird absteigend nach der Höhe der values sortiert 
(die häufigst vorkommenden Wortformen werden an den Anfang gestellt).
'''
keyword_dict_complete = {k: v for k, v in sorted(keyword_dict_complete.items(), key=lambda item: item[1], reverse=True)}

'''
Das Dictionary wird in einen Dataframe überführt 
und das Ergebnis abschließend in einer csv-Datei gespeichert.
'''
df_keyword = pd.DataFrame.from_dict(keyword_dict_complete, orient='index', columns=["Häufigkeit"])
df_keyword.to_csv(f"{keyword}_einzelformen.csv")

Das Ergebnis dieser Analyse lässt sich auch durch Nutzung des Ergebnis-Dictionary keyword_dict_complete und der Python-Library WordCloud in einer Wortwolke darstellen. Für die Funktionalitäten von WordCloud siehe auch https://www.python-lernen.de/wordcloud-erstellen-python.htm. Generell bietet sich diese Form der Visualisierung für verschiedene hier durchgeführte Textanalyseschritte an.

In [ ]:
''' 
Erstellen der WordCloud und Speichern als png
'''
wc = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(keyword_dict_complete)
plt.figure(figsize=(15, 5))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.savefig(f"{keyword}_wortwolke.png", facecolor='k', bbox_inches='tight')

### Identifizierung von Einzeltexten, die einen Suchbegriff enthalten <a class="anchor" id="einzeltexte"></a>

Das nachfolgende Skript soll die Möglichkeit bieten, die Anzahl und die Namen von Einzeltexten herauszufiltern, die einen bestimmten Suchbegriff enthalten. Während die vorangehenden Analyseschritte darauf basieren, dass die Texte nach Jahren gruppiert werden, um eine sinnvolle Auswertung zu ermöglichen, muss das nachfolgende Skript auf den aus media/rep/ extrahierten Einzeltexten ausgeführt werden. Hierzu ist prinzipiell keine Vorbearbeitung der Texte notwendig, es muss jedoch mit einer längeren Durchlaufzeit des Skriptes gerechnet werden (ca. 4-6 Minuten). Möchte man die Durchlaufzeit des Skriptes beschleunigen, kann das oben beschriebene Filtern von Stoppwörtern auch auf den Einzeldateien durchgeführt werden, was allerdings ebenfalls einen gewissen Zeitaufwand mit sich bringt. 
  
Die nachfolgende Analyse stützt sich auf absolute im Gegensatz zu relativen Zahlen. Dies bedeutet, dass die Häufigkeit des Vorkommens des Suchbegriffes in einem Text nicht in Bezug zur Gesamtanzahl der Wörter des Textes gesetzt wird. Die Nutzung des Skriptes bietet sich an, wenn man gezielt nach Publikationen zu einem bestimmten Begriff sucht. 
  
Das Skript sieht die Suche nach einem einzelnen Begriff vor und nicht die Suche nach mehreren Begriffen auf einmal. Hierzu müsste das Skript mehrmals hintereinander mit jeweils einem neuen Suchbegriff durchlaufen werden.

In [4]:
'''
Eingabe des Suchbegriffes, 
für den das Vorkommen in Einzeltexten ermittelt wird.
Mit dieser Suche kann nicht nur nach vollständigen Wörtern gesucht werden. 
Es werden auch Wörter wiedergegeben, 
die mit dem eingegegeben Suchbegriff beginnen, 
aber unterschiedliche Endungen aufweisen 
Beispiel:
Für "digital" wird auch "digitalität", "digitales", "digitally" wiedergegeben.
''' 
keyword_singletexts = "" #z.B. "virtuell"

'''
Einzufügen ist der Name des Dateiordners, 
der die zu analysierenden txt-Dateien enthält. 
In diesem Ordner müssen die aus media/rep/ extrahierten Einzeldateien vorliegen
und keine nach Jahren gruppierten Texten.
'''
folder = "" #z.B. "fulltext"

In [ ]:
'''
Anlegen eines leeren Dataframe, 
um die Ergebnisse der Häufigkeitsermittlung festzuhalten.
'''
df_singletexts = pd.DataFrame()

'''
Anlegen einer leeren Liste, 
in der die Namen der Einzeltexte gespeichert werden, 
die als Index des erstellten Dataframe fungieren.
'''
filename_singletexts_list =  []

'''
Anlegen einer leeren Liste, 
in der die im Weiteren erstellten Dictionaries gesammelt werden.
'''
list_singletexts_keyword = []

'''
Jede im Dateiordner vorliegende txt-Datei wird nacheinander eingelesen 
und die Texte zur besseren Vergleichbarkeit 
vollständig in Kleinbuchstaben umgewandelt.
'''
for file in glob.glob(os.path.join(f"{folder}", "*.txt")):
    with open(file, "r", encoding="utf-8") as infile:
        text_singletexts = infile.read().lower()

        '''
        Jeder Text wird von einem String in eine Wortliste umgewandelt, 
        wobei die Wörter an den Nicht-Wortzeichen gesplittet werden.
        '''
        text_singletexts_list = re.split("\W+", text_singletexts)

        '''
        Die Dateinamen der eingelesenen Texte 
        werden auf ihren Kernnamen reduziert 
        (Dateierweiterung, Dateipfad wird entfernt) 
        und in einer Liste gesammelt, 
        die im Weiteren den Index des Dataframes darstellt.
        '''
        filename_singletexts = Path(file).stem
        filename_singletexts_list.append(filename_singletexts)

        '''
        Erstellen eines Dictionary für jeden Text.
        Hierzu wird über die Wortliste eines jeden Textes iteriert 
        und die gefundenen Suchbegriffe als keys dem Dictionary hinzugefügt.
        Bei erstmaligem Vorkommen erhält der Suchbegriff 1 als value.
        Taucht der Suchbegriff mehrfach im Text auf, 
        wird der value für jedes Vorkommen hochgezählt.
        '''
        singletexts_dict = {}
        for i in text_singletexts_list:
                if i in singletexts_dict:
                    singletexts_dict[i] +=1
                else:
                    singletexts_dict[i] = 1
        
        '''
        Aus den für die einzelnen Texte erstellen Dictionaries 
        werden nur die Wörter ausgewählt, 
        die mit den als keywords eingetragenen Suchbegriffen beginnen.
        Die Anzahl des Vorkommens der einzelnen Wortformen und -kombinationen 
        zu einem Suchbegriff wird addiert. 
        Das Ergebnis wird einer Liste hinzugefügt.
        '''
        keyword_singletexts_dict = {k: v for k, v in singletexts_dict.items() if k.startswith(keyword_singletexts)}
        list_singletexts_keyword.append(sum(keyword_singletexts_dict.values()))

'''
Die Ergebnisse aus dem Dictionary sowie die Anzahl der Wörter pro Text 
werden in den Dataframe eingefügt.
'''
df_singletexts.index = filename_singletexts_list
df_singletexts[keyword_singletexts] = list_singletexts_keyword

'''
Ausgabe der Anzahl der Texte des Gesamtkorpus, 
die den Suchbegriff enthalten, sowie den Prozentanteil, 
den diese Texte vom Gesamtkorpus ausmachen.
'''
text_with_content = len(list_singletexts_keyword) - list_singletexts_keyword.count(0)
percent_content = round(text_with_content / len(list_singletexts_keyword) * 100, 2)
print(f"Anzahl der Texte, die den Suchbegriff '{keyword_singletexts}' enthalten: {text_with_content} (in Prozent: {percent_content}%) \n")

'''
Gibt die 10 häufigsten Textnamen 
und die Häufigkeit der Nennung in absoluten Zahlen der Texte aus, 
die den Suchbegriff am häufigsten enthalten
'''
print(f"Nachfolgend sind die 10 Texte gelistet, in denen der Suchbegriff '{keyword_singletexts}' in absoluten Zahlen am häufigsten vorkommt: \n")

print(df_singletexts[keyword_singletexts].nlargest(10).to_string())